In [ ]:
# default_exp tree_evaluator

# tree_evaluator
> Tree evaluator.

In [ ]:
# export
from uid3.att_stats import AttStats
from uid3.attribute import Attribute
from uid3.data import Data
from uid3.tree import Tree
from uid3.uncertain_entropy_evaluator import UncertainEntropyEvaluator

In [ ]:
# export
class TreeEvaluator:
    def __init__(self):
        pass

In [ ]:
# export
class BenchmarkResult:
    def __init__(self, class_attribute: Attribute):
        self.stats = []
        self.predicitons = []
        
        self.stats = [class_label for class_label in class_attribute.get_domain()]
    
    def train_and_test(self, training_data: Data, test_data: Data):
        trained_tree = UId3.grow_tree(trainingData, UncertainEntropyEvaluator(), 0)
        
        return self.test(trained_tree, test_data)


    def test(trained_tree: Tree, test_data: Data):
        result = BenchmarkResult(test_data.get_class_attribute())

        for i in test_data.get_instances():
            prediction = trained_tree.predict(i)
            error =  not prediction.get_most_porbable().get_name() == i.get_readings().get_last().get_most_probable().get_name()
            result.add_prediction(Prediction(prediction, i.get_readings().get_last().get_most_probable().get_name()))
            if error:
                #give false positive to predicted class, false negative to real class, and true negatives to other
                predicted_name = prediction.get_most_porbable().get_name()
                real_name = i.get_readings().get_last().get_most_probable().get_name()
                result.add_FP(predicted_name)
                result.add_FN(real_name)
                for s in result.stats: 
                    if not s.class_label == predicted_name and not s.class_label == real_name:
                        result.add_TN(s.get_class_label())
                result.incorrect += 1 
            else:
                #add true positive for predicted class, and true negatives for other
                predicted_name = prediction.get_most_porbable().get_name()
                result.add_TP(predicted_name)
                for s in result.stats:
                    if not s.class_label == predicted_name:
                        result.add_TN(s.get_class_label())
                result.correct += 1
        return result

    def get_predictions(self) -> list:
        return predictions
    
    def get_accuracy(self) -> float:
        return correct / (correct + incorrect)
    
    def get_stats_for_label(self, class_label: str) -> Stats: 
        for s in self.stats:
            if s.get_class_label() == class_label:
                return s
        return None
    
    def add_TP(self, value: str) -> None:
        for s in self.stats:
            if s.get_class_label() == value: 
                s.set_TP(s.get_TP() + 1)
                break

    def add_FP(self, value: str) -> None:
        for s in self.stats:
            if s.get_class_label() == value: 
                s.set_FP(s.get_FP() + 1)
                break

    def add_TN(self, value: str) -> None:
        for s in self.stats:
            if s.get_class_label() == value: 
                s.set_TN(s.get_TN() + 1)
                break
   
    def add_FN(self, value: str) -> None:
        for s in self.stats:
            if s.get_class_label() == value: 
                s.set_FN(s.get_FN() + 1)
                break

    def add_prediction(self, prediction: Prediction) -> None:
        predictions.append(prediction)


In [ ]:
# export
class Prediction:
    def __init__(self, prediction: AttStats, correct_label: str):
        self.prediction = prediction
        self.correct_label = correct_label
        
    def __lt__(self, p1):
        prob1 = self.prediction.get_stat_for_value(class_label)
        prob2 = p1.prediction.get_stat_for_value(class_label)
        return prob1 < prob2
    
    def __le__(self, p1):
        prob1 = self.prediction.get_stat_for_value(class_label)
        prob2 = p1.prediction.get_stat_for_value(class_label)
        return prob1 <= prob2
    
    def __eq__(self, p1): 
        prob1 = self.prediction.get_stat_for_value(class_label)
        prob2 = p1.prediction.get_stat_for_value(class_label)
        return prob1 == prob2
    
    def __ne__(self, p1): 
        prob1 = self.prediction.get_stat_for_value(class_label)
        prob2 = p1.prediction.get_stat_for_value(class_label)
        return prob1 != prob2
    
    def __gt__(self, p1): 
        prob1 = self.prediction.get_stat_for_value(class_label)
        prob2 = p1.prediction.get_stat_for_value(class_label)
        return prob1 > prob2
    
    def __ge__(self, p1): 
        prob1 = self.prediction.get_stat_for_value(class_label)
        prob2 = p1.prediction.get_stat_for_value(class_label)
        return prob1 >= prob2
            

In [ ]:
# export
class Stats:
    def __init__(self, class_label):
        self.class_label = class_label
        
    def get_TP_rate(self) -> float:
        return self.get_TP() / (self.get_TP() + self.get_FN())

    def get_FP_rate(self) -> float:
        return self.get_FP() / (self.get_FP() + self.get_TN())

    def get_precision(self) -> float:
        return self.get_TP() / (self.get_TP() + self.get_FP())

    def get_recall(self) -> float:
        return self.get_TP() / (self.get_TP() + self.get_FN())

    def get_F_measure(self) -> float:
        return 2 * (self.getPrecision() * self.getRecall()) / (self.getPrecision() + self.getRecall())

    def get_ROC_area(bench: BenchmarkResult) -> float:
        result = 0
        preds = bench.get_predictions()
        
        preds = sorted(preds) 

        n_class_count = 0
        y_class_count = 0
        uy  = 0
        un = 0
        for p in preds:
            if p.correct_label == class_label:
                uy += n_class_count
                y_class_count += 1
            else:
                un += y_class_count
                n_class_count += 1

        result = uy / (uy + un)
        return result

    def get_class_label(self) -> str:
        return class_label

    def set_class_label(class_label: str) -> None:
        self.class_label = class_label

    def get_TP(self) -> float:
        return TP

    def set_TP(self, TP: float) -> None:
        self.TP = TP

    def get_FP(self) -> float:
        return FP

    def set_FP(self, FP: float) -> None:
        self.FP = FP

    def get_TN(self) -> float:
        return TN

    def set_TN(self, TN: float) -> None:
        self.TN = TN

    def get_FN(self) -> float:
        return FN

    def set_FN(self, FN: float) -> None:
        self.FN = FN


In [ ]:
class Testabc:
    def __init__(self, prediction: int):
        self.prediction = prediction
        
    def __lt__(self, p1):
        prob1 = self.prediction
        prob2 = p1.prediction
        return prob1 < prob2
    
    def __le__(self, p1):
        prob1 = self.prediction
        prob2 = p1.prediction
        return prob1 <= prob2
    
    def __eq__(self, p1): 
        prob1 = self.prediction
        prob2 = p1.prediction
        return prob1 == prob2
    
    def __ne__(self, p1): 
        prob1 = self.prediction
        prob2 = p1.prediction
        return prob1 != prob2
    
    def __gt__(self, p1): 
        prob1 = self.prediction
        prob2 = p1.prediction
        return prob1 > prob2
    
    def __ge__(self, p1): 
        prob1 = self.prediction
        prob2 = p1.prediction
        return prob1 >= prob2
    
    def __str__(self):
        return str(self.prediction)

In [ ]:
test1 = Testabc(3)
test2 = Testabc(63)
test3 = Testabc(33)
test4 = Testabc(87)

In [ ]:
sorted([test1, test2, test3, test4])

In [ ]:
for a in sorted([test1, test2, test3, test4]):
    print(str(a))

3
33
63
87


In [ ]:
test1 <test2

True